# Gene-to-graph workflow

This notebook showcases the steps to generate the BioDataFuse data and graph serializations from a list of genes.


In [1]:
# Setting up the working directory
import os
import time

start_time = time.time()
current_dir = os.getcwd()

os.chdir(os.path.join(f"{current_dir}", ".."))

# Set the current working directory
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /Users/yojana/Documents/GitHub/pyBiodatafuse


In [2]:
# Import modules
import pickle

import pandas as pd
import numpy as np
from IPython.display import Image, display

from pyBiodatafuse import id_mapper
from pyBiodatafuse.annotators import (
    bgee,
    disgenet,
    minerva,
    molmedb,
    opentargets,
    pubchem,
    stringdb,
    wikipathways,
)
from pyBiodatafuse.constants import DISGENET_DISEASE_COL
from pyBiodatafuse.graph import saver
from pyBiodatafuse.graph.rdf import BDFGraph
from pyBiodatafuse.utils import (
    combine_sources,
    create_harmonized_input_file,
    create_or_append_to_metadata,
)

/opt/anaconda3/envs/pybiodatafuse/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Entity resolution with BridgeDB

The first step is to input the list of genes to query and retrieve their protein target and synonym identifiers using BridgeDB.

### 1.1. Load the input list and convert it to a dataframe

In [3]:
genes_of_interest = """AAGRN
ALG14
ALG2
CHAT
CHD8
CHRNA1
CHRNB1
CHRND
CHRNE
CHRNG
COL13A1
COLQ
DOK7
DPAGT1
GFPT1
GMPPB
LAMA5
LAMB2
LRP4
MUSK
MYO9A
PLEC
PREPL
PURA
RAPSN
RPH3A
SCN4A
SLC18A3
SLC25A1
SLC5A7
SNAP25
SYT2
TOR1AIP1
UNC13A
VAMP1
DMD"""

gene_list = genes_of_interest.split("\n")
data_input = pd.DataFrame(gene_list, columns=["identifier"])
data_input.head()

,identifier
0,AAGRN
1,ALG14
2,ALG2
3,CHAT
4,CHD8


### 1.2. Query BridgeDB
The results will be stored in the following directories:

In [4]:
base_dir = os.path.abspath(os.getcwd())  # Ensures an absolute path
DATA_DIR = os.path.join(base_dir, "examples", "data")
EXAMPLE_DIR = os.path.join(base_dir, "examples", "data", "gene_to_graph_workflow")
os.makedirs(DATA_DIR, exist_ok=True)  # TODO paths
os.makedirs(EXAMPLE_DIR, exist_ok=True)

In [5]:
pickle_path = f"{EXAMPLE_DIR}/example_gene_list.pkl"
metadata_path = f"{EXAMPLE_DIR}/example_gene_list_metadata.pkl"

if not os.path.exists(pickle_path):
    bridgedb_df, bridgedb_metadata = id_mapper.bridgedb_xref(
        identifiers=data_input,
        input_species="Human",
        input_datasource="HGNC",
        output_datasource="All",
    )
    bridgedb_df.to_pickle(pickle_path)
    with open(metadata_path, "wb") as file:
        pickle.dump(bridgedb_metadata, file)
else:
    bridgedb_df = pd.read_pickle(pickle_path)
    with open(metadata_path, "rb") as file:
        bridgedb_metadata = pickle.load(file)

Alternatively, you can also skip `input_datasource` for most identifier types:

In [6]:
bridgedb_df_test, bridgedb_metadata_test = id_mapper.bridgedb_xref(
    identifiers=data_input,
    input_species="Human",
    # input_datasource="HGNC", # Commented out
    output_datasource="All",
)
bridgedb_df_test

,identifier,identifier.source,target,target.source
0,ALG14,HGNC,1553954_at,Affy
1,ALG14,HGNC,Q96F25,Uniprot-TrEMBL
2,ALG14,HGNC,GO:0016021,Gene Ontology
3,ALG14,HGNC,GO:0016020,Gene Ontology
4,ALG14,HGNC,A_23_P257423,Agilent
...,...,...,...,...
4692,DMD,HGNC,4004045,Affy
4693,DMD,HGNC,4004287,Affy
4694,DMD,HGNC,4004286,Affy
4695,DMD,HGNC,4004285,Affy


In [7]:
print("Number of genes with mapping in BridgeDb:", len(bridgedb_df["identifier"].unique()))
bridgedb_df.head()

Number of genes with mapping in BridgeDb: 35


,identifier,identifier.source,target,target.source
0,ALG14,HGNC,1553954_at,Affy
1,ALG14,HGNC,Q96F25,Uniprot-TrEMBL
2,ALG14,HGNC,GO:0016021,Gene Ontology
3,ALG14,HGNC,GO:0016020,Gene Ontology
4,ALG14,HGNC,A_23_P257423,Agilent


## 2. Gene expression
### 2.1. Gene expression from Bgee

In [ ]:
bgee_path = f"{EXAMPLE_DIR}/example_bgee.pkl"
bgee_metadata_path = f"{EXAMPLE_DIR}/example_bgee_metadata.pkl"

if not os.path.exists(bgee_path):
    bgee_df, bgee_metadata = bgee.get_gene_expression(bridgedb_df=bridgedb_df)
    bgee_df.to_pickle(bgee_path)
    with open(bgee_metadata_path, "wb") as file:
        pickle.dump(bgee_metadata, file)
else:
    bgee_df = pd.read_pickle(bgee_path)
    with open(bgee_metadata_path, "rb") as file:
        bgee_metadata = pickle.load(file)

Querying Bgee:   2%|▏         | 4/169 [03:39<2:31:27, 55.08s/it]

## 3. Disease annotation
### 3.1. Gene to disease annotation with DisGeNET

In [9]:
disgenet_path = f"{EXAMPLE_DIR}/example_disgenet.pkl"
disgenet_metadata_path = f"{EXAMPLE_DIR}/example_disgenet_metadata.pkl"

disgenet_api_key = "PASTE_YOUR_DISGENET_API_KEY_HERE"

if not os.path.exists(disgenet_path):
    disgenet_df, disgenet_metadata = disgenet.get_gene_disease(
        api_key=disgenet_api_key, bridgedb_df=bridgedb_df
    )

    disgenet_df.to_pickle(disgenet_path)
    with open(disgenet_metadata_path, "wb") as file:
        pickle.dump(disgenet_metadata, file)
else:
    disgenet_df = pd.read_pickle(disgenet_path)
    with open(disgenet_metadata_path, "rb") as file:
        disgenet_metadata = pickle.load(file)

### 3.2. Disease to compound annotation from OpenTargets

In [10]:
# Prepare the input to use DISGENET output as seed for OpenTargets
disease_mapping_df = create_harmonized_input_file(disgenet_df, DISGENET_DISEASE_COL, "EFO", "UMLS")
disease_mapping_df.head()

,identifier,identifier.source,target,target.source
0,UMLS_C5680989,UMLS,EFO_0700079,EFO
2,UMLS_C0017605,UMLS,EFO_1001506,EFO
3,UMLS_C0751884,UMLS,EFO_0700127,EFO
4,UMLS_C0011581,UMLS,EFO_0004257,EFO
5,UMLS_C0011581,UMLS,EFO_0008623,EFO


In [11]:
opentarget_path = f"{EXAMPLE_DIR}/example_opentarget_cmpd.pkl"
opentarget_metadata_path = f"{EXAMPLE_DIR}/example_opentarget_cmpd_metadata.pkl"

if not os.path.exists(opentarget_path):
    opentargets_df, opentargets_metadata = opentargets.get_disease_compound_interactions(
        disease_mapping_df
    )
    opentargets_df.to_pickle(opentarget_path)
    with open(opentarget_metadata_path, "wb") as file:
        pickle.dump(opentargets_metadata, file)
else:
    opentargets_df = pd.read_pickle(opentarget_path)
    with open(opentarget_metadata_path, "rb") as file:
        opentargets_metadata = pickle.load(file)

## 4. Pathways and Gene Ontology terms
### 4.1. Pathways from MINERVA

In [12]:
minerva_path = f"{EXAMPLE_DIR}/example_minerva.pkl"
minerva_metadata_path = f"{EXAMPLE_DIR}/example_minerva_metadata.pkl"


if not os.path.exists(minerva_path):
    minerva_df, minerva_metadata = minerva.get_gene_minerva_pathways(
        bridgedb_df, map_name="COVID19 Disease Map"
    )
    minerva_df.to_pickle(minerva_path)
    with open(minerva_metadata_path, "wb") as file:
        pickle.dump(minerva_metadata, file)

else:
    minerva_df = pd.read_pickle(minerva_path)
    with open(minerva_metadata_path, "rb") as file:
        minerva_metadata = pickle.load(file)

### 4.2. Pathways from WikiPathways

In [13]:
wikipathways_path = f"{EXAMPLE_DIR}/example_wikipathway.pkl"
wikipathways_metadata_path = f"{EXAMPLE_DIR}/example_wikipathway_metadata.pkl"

if not os.path.exists(wikipathways_path):
    wikipathways_df, wikipathways_metadata = wikipathways.get_gene_wikipathways(
        bridgedb_df=bridgedb_df
    )
    wikipathways_df.to_pickle(wikipathways_path)
    with open(wikipathways_metadata_path, "wb") as file:
        pickle.dump(wikipathways_metadata, file)

else:
    wikipathways_df = pd.read_pickle(wikipathways_path)
    with open(wikipathways_metadata_path, "rb") as file:
        wikipathways_metadata = pickle.load(file)

### 4.3. Reactome pathways from OpenTargets

In [14]:
opentargets_reactome_path = f"{EXAMPLE_DIR}/example_ot_reactome.pkl"
opentargets_reactome_metadata_path = f"{EXAMPLE_DIR}/example_ot_reactome_metadata.pkl"

if not os.path.exists(opentargets_reactome_path):
    opentargets_reactome_df, opentargets_reactome_metadata = opentargets.get_gene_reactome_pathways(
        bridgedb_df=bridgedb_df
    )
    opentargets_reactome_df.to_pickle(opentargets_reactome_path)
    with open(opentargets_reactome_metadata_path, "wb") as file:
        pickle.dump(opentargets_reactome_metadata, file)
else:
    opentargets_reactome_df = pd.read_pickle(opentargets_reactome_path)
    with open(opentargets_reactome_metadata_path, "rb") as file:
        opentargets_reactome_metadata = pickle.load(file)

### 4.4. Gene Ontology from OpenTargets

In [15]:
opentargets_go_path = f"{EXAMPLE_DIR}/example_ot_go.pkl"
opentargets_go_metadata_path = f"{EXAMPLE_DIR}/example_ot_go_metadata.pkl"

if not os.path.exists(opentargets_go_path):
    opentargets_go_df, opentargets_go_metadata = opentargets.get_gene_go_process(
        bridgedb_df=bridgedb_df
    )
    opentargets_go_df.to_pickle(opentargets_go_path)
    with open(opentargets_go_metadata_path, "wb") as file:
        pickle.dump(opentargets_go_metadata, file)
else:
    opentargets_go_df = pd.read_pickle(opentargets_go_path)
    with open(opentargets_go_metadata_path, "rb") as file:
        opentargets_go_metadata = pickle.load(file)

## 5. Compound annotation
### 5.1. Compound annotation from OpenTargets

In [16]:
opentargets_gene_path = f"{EXAMPLE_DIR}/example_ot_gene_cmpd.pkl"
opentargets_gene_metadata_path = f"{EXAMPLE_DIR}/example_ot_gene_cmpd_metadata.pkl"

if not os.path.exists(opentargets_gene_path):
    opentargets_compound_df, opentargets_compound_metadata = (
        opentargets.get_gene_compound_interactions(bridgedb_df=bridgedb_df)
    )
    opentargets_compound_df.to_pickle(opentargets_gene_path)
    with open(opentargets_gene_metadata_path, "wb") as file:
        pickle.dump(opentargets_compound_metadata, file)

else:
    opentargets_compound_df = pd.read_pickle(opentargets_gene_path)
    with open(opentargets_gene_metadata_path, "rb") as file:
        opentargets_compound_metadata = pickle.load(file)

### 5.2. Screening results of compounds on proteins encoded by genes annotation by PubChem

In [17]:
pubchem_path = f"{EXAMPLE_DIR}/example_pubchem.pkl"
pubchem_metadata_path = f"{EXAMPLE_DIR}/example_pubchem_metadata.pkl"

if not os.path.exists(pubchem_path):
    pubchem_assay_df, pubchem_assay_metadata = pubchem.get_protein_compound_screened(
        bridgedb_df=bridgedb_df
    )
    pubchem_assay_df.to_pickle(pubchem_path)
    with open(pubchem_metadata_path, "wb") as file:
        pickle.dump(pubchem_assay_metadata, file)

else:
    pubchem_assay_df = pd.read_pickle(pubchem_path)
    with open(pubchem_metadata_path, "rb") as file:
        pubchem_assay_metadata = pickle.load(file)

## 6. Membrane transport annotations
### 6.1 Transporter inhibitor annotation from MolMeDB

In [18]:
molmedb_path = f"{EXAMPLE_DIR}/example_molmedb_gene_cmpd.pkl"
molmedb_metadata_path = f"{EXAMPLE_DIR}/example_molmedb_gene_cmpd_metadata.pkl"

if not os.path.exists(molmedb_path):
    inhibitor_df, inhibitor_metadata = molmedb.get_gene_compound_inhibitor(bridgedb_df=bridgedb_df)
    inhibitor_df.to_pickle(molmedb_path)
    with open(molmedb_metadata_path, "wb") as file:
        pickle.dump(inhibitor_metadata, file)
else:
    inhibitor_df = pd.read_pickle(molmedb_path)
    with open(molmedb_metadata_path, "rb") as file:
        inhibitor_metadata = pickle.load(file)

## 7. Protein-Protein Interactions

### 7.1. Protein-Protein Interactions from STRING

In [19]:
string_path = f"{EXAMPLE_DIR}/example_string.pkl"
string_metadata_path = f"{EXAMPLE_DIR}/example_string_metadata.pkl"

if not os.path.exists(string_path):
    ppi_df, ppi_metadata = stringdb.get_ppi(bridgedb_df=bridgedb_df)
    ppi_df.to_pickle(string_path)
    with open(string_metadata_path, "wb") as file:
        pickle.dump(ppi_metadata, file)
else:
    ppi_df = pd.read_pickle(string_path)
    with open(string_metadata_path, "rb") as file:
        ppi_metadata = pickle.load(file)

## 8. Graph generation 

### 8.1. Combine all data and metadata

In [20]:
combined_df = combine_sources(
    bridgedb_df,
    [
        bgee_df,
        disgenet_df,
        minerva_df,
        wikipathways_df,
        opentargets_reactome_df,
        opentargets_go_df,
        opentargets_compound_df,
        inhibitor_df,
        pubchem_assay_df,
        ppi_df,
    ],
)

In [21]:
combined_df.to_pickle(f"{EXAMPLE_DIR}/combined_df.pkl")

The following code chunk shuffles the DisGeNET-queried data before serializing the graphs and displaying them, as it is not open access.

In [22]:
# Shuffling the rows
combined_df["DISGENET_diseases"] = combined_df["DISGENET_diseases"].apply(np.random.permutation)

combined_metadata = create_or_append_to_metadata(
    bridgedb_metadata,
    [
        bgee_metadata,
        disgenet_metadata,
        opentargets_metadata,
        opentargets_compound_metadata,
        inhibitor_metadata,
        pubchem_assay_metadata,
        ppi_metadata,
        wikipathways_metadata,
        minerva_metadata,
        opentargets_reactome_metadata,
        opentargets_go_metadata,
    ],
)

We export the combined (meta) data in pickle format:

In [23]:
combined_df.to_pickle(f"{EXAMPLE_DIR}/example_df_shuffled.pkl")
with open(f"{EXAMPLE_DIR}/example_metadata.pkl", "wb") as out:
    pickle.dump(combined_metadata, out)

### 8.2. Create a graph from the annotated dataframe

In [24]:
pygraph = saver.save_graph(
    combined_df=combined_df,
    combined_metadata=combined_metadata,
    disease_compound=opentargets_df,
    graph_name="gene_examples",
    graph_dir=EXAMPLE_DIR,
)

Combined DataFrame saved in /Users/yojana/Documents/GitHub/pyBiodatafuse/examples/data/gene_to_graph_workflow/gene_examples_df.pkl
Metadata saved in /Users/yojana/Documents/GitHub/pyBiodatafuse/examples/data/gene_to_graph_workflow/gene_examples_metadata.pkl
Building graph: 100%|██████████| 35/35 [00:00<00:00, 4886.19it/s]
Graph is built successfully
Graph saved in /Users/yojana/Documents/GitHub/pyBiodatafuse/examples/data/gene_to_graph_workflow/gene_examples_graph.pkl and /Users/yojana/Documents/GitHub/pyBiodatafuse/examples/data/gene_to_graph_workflow/gene_examples_graph.gml
Graph saved in /Users/yojana/Documents/GitHub/pyBiodatafuse/examples/data/gene_to_graph_workflow/gene_examples_graph.edgelist


In [26]:
for node, ndata in pygraph.nodes(data=True):
    if not ndata:
        print(node)

    # try:
    #     if "UMLS" in node:
    #         print(node, ndata)
    # except:
    #     print(node, ndata)
    #     break

UMLS:C0000786
UMLS:C0001125
UMLS:C0003089
UMLS:C0003469
UMLS:C0003873
UMLS:C0004352
UMLS:C0006142
UMLS:C0006663
UMLS:C0007193
UMLS:C0007194
UMLS:C0008073
UMLS:C0009402
UMLS:C0009404
UMLS:C0011581
UMLS:C0012242
UMLS:C0013080
UMLS:C0013404
UMLS:C0014544
UMLS:C0014547
UMLS:C0014549
UMLS:C0016059
UMLS:C0017668
UMLS:C0018800
UMLS:C0018817
UMLS:C0022658
UMLS:C0023269
UMLS:C0025281
UMLS:C0025322
UMLS:C0026848
UMLS:C0026896
UMLS:C0027540
UMLS:C0027726
UMLS:C0028754
UMLS:C0029516
UMLS:C0030193
UMLS:C0031117
UMLS:C0032227
UMLS:C0032290
UMLS:C0032460
UMLS:C0033141
UMLS:C0035126
UMLS:C0035229
UMLS:C0035309
UMLS:C0035410
UMLS:C0036439
UMLS:C0036529
UMLS:C0036572
UMLS:C0037772
UMLS:C0038220
UMLS:C0038356
UMLS:C0041696
UMLS:C0206656
UMLS:C0264886
UMLS:C0339573
UMLS:C0376358
UMLS:C0393541
UMLS:C0522224
UMLS:C0525045
UMLS:C0878544
UMLS:C1260409
UMLS:C1269683
UMLS:C1449563
UMLS:C1510586
UMLS:C1535926
UMLS:C1862939
UMLS:C3554373
UMLS:C5681306


In [27]:
for source, target, data in pygraph.edges(data=True):
    if source == "UMLS:C0000786" or target == "UMLS:C0000786":
        print(source, target, data)
        break

pubchem.compound:55245 UMLS:C0000786 {'label': 'treats', 'datasource': 'OpenTargets'}


#### 8.3. Cytoscape
Make sure that the Cytoscape is open.

In [ ]:
# from pyBiodatafuse.graph import cytoscape

# cytoscape.load_graph(pygraph, network_name="Test network")

#### 8.4. Neo4j

In [ ]:
# from pyBiodatafuse.graph import neo4j

# neo4j.save_graph_to_graphml(pygraph, "networkx_graph_test.graphml")

##### Steps to load the graph in Neo4j

- Add `.graphml` file in **import** subfolder of the DBMS folder
- Install apoc plugin
- Create `apoc.conf` file:
    ```
    apoc.trigger.enabled=true
    apoc.import.file.enabled=true
    apoc.export.file.enabled=true
    apoc.import.file.use_neo4j_config=true
    ```
- Add `apoc.conf` file to **conf** subfolder of the DBMS folder
- Open Neo4j Browser
- (Optionl, only run if you have imported a graph  before) Remove all the nodes before importing `.graphml` file

    ```MATCH (n) DETACH DELETE n```

- Import `.graphml` file

    ```call apoc.import.graphml('file:///networkx_graph_test.graphml',{readLabels:TRUE})```

- Add indexes after importing the graph for improving the performance of queries

    ```
    create index Gene for (n:Gene) on (n.node_type)
    create index Pathway for (n:Pathway) on (n.node_type)
    create index `Biological Process` for (n:`Biological Process`) on (n.node_type)
    create index `Molecular Function` for (n:`Molecular Function`) on (n.node_type)
    create index `Cellular Component` for (n:`Cellular Component`) on (n.node_type)
    create index Disease for (n:Disease) on (n.node_type)
    create index Compound for (n:Compound) on (n.node_type)
    create index `Side Effect` for (n:`Side Effect`) on (n.node_type)
    ```
    

- Count the number of each node type
    - total (```MATCH (n) RETURN count(n)```) 
        - Gene (```MATCH (n:Gene) RETURN count(n)```)
        - Pathway (```MATCH (n:Pathway) RETURN count(n)```)
            - WikiPathways (```MATCH (n:Pathway {source: "WikiPathways"}) RETURN count(n)```) 
            - OpenTargets, Reactome (```MATCH (n:Pathway {source: "OpenTargets"}) RETURN count(n)```) 
            - MINERVA (```MATCH (n:Pathway {source: "MINERVA"}) RETURN count(n)```) 
        - Biological Process (```MATCH (n:`Biological Process`) RETURN count(n)```) 
        - Molecular Function (```MATCH (n:`Molecular Function`) RETURN count(n)```) 
        - Cellular Component (```MATCH (n:`Cellular Component`) RETURN count(n)```) 
        - Disease (```MATCH (n:Disease) RETURN count(n)```) 
        - Compound (```MATCH (n:Compound) RETURN count(n)```)
        - Side Effect (```MATCH (n:`Side Effect`) RETURN count(n)```) 
- Count the number of each edge type
    - total (```MATCH ()-[r]->() RETURN count(r)```) 
        - interacts_with (```MATCH ()-[r:interacts_with]->() RETURN count(r)```) 
        - part_of (```MATCH ()-[r:part_of]->() RETURN count(r)```) 
            - WikiPathways (```MATCH ()-[r:part_of {source: "WikiPathways"}]->() RETURN count(r)```) 
            - OpenTargets, Reactome (```MATCH ()-[r:part_of {source: "OpenTargets"}]->() RETURN count(r)```) 
            - MINERVA (```MATCH ()-[r:part_of {source: "MINERVA"}]->() RETURN count(r)```) 
        - activates (```MATCH ()-[r:activates]->() RETURN count(r)```) 
        - treats (```MATCH ()-[r:treats]->() RETURN count(r)```) 
        - has_side_effect (```MATCH ()-[r:has_side_effect]->() RETURN count(r)```) 
        - inhibits (```MATCH ()-[r:inhibits]->() RETURN count(r)```) = 71
        - associated_with (```MATCH ()-[r:associated_with]->() RETURN count(r)```) 

- Export the graph as a `.csv` file

    ```call apoc.export.csv.all("networkx_graph_test.csv",{})```

#### 8.5. GraphDB (RDF)

In [ ]:
# Instantiate a BDFGraph
# bdf = BDFGraph(
#     base_uri="https://biodatafuse.org/example/",
#     version_iri="https://biodatafuse.org/example/test.owl",
#     orcid="https://orcid.org/0000-0002-4166-7093",
#     author="Javier Millan Acosta",
# )

# bdf.generate_rdf(combined_df, combined_metadata)  # Generate the RDF from the (meta)data

In [ ]:
# bdf.serialize(
#     os.path.join(base_dir, "examples", "data", "gene_to_graph_workflow", "BDF_example_graph.ttl"),
#     format="ttl",
# )

##### 8.5.1. Generate prefixes SHACL

SHACL graphs defining namespaces and prefixes can be loaded into SPARQL endpoints to avoid having to declare prefixes in the query.

In [ ]:
# Use without parameters (defaults, does not save file)
# bdf.shacl_prefixes()
# # or Use with parameters
# bdf.shacl_prefixes(
#     path=os.path.join(base_dir, "examples", "data", "gene_to_graph_workflow", "BDF_prefixes.ttl"),
#     namespaces=None,  # Optional, add more namespaces with a dictionary of {prefix:namespace,}
# )

##### 8.5.2. Use [`shexer`](https://github.com/DaniFdezAlvarez/shexer/) to retrieve the RDF shapes

The `shexer` library is used to retrieve the shapes of the graph in SHACL (https://www.w3.org/TR/shacl/) and ShEx (https://shex.io/shex-semantics/).

- **SHACL**

In [ ]:
# # Use without parameters (defaults)
# # bdf.shacl()

# # Or use with parameters
# bdf.shacl(
#     path=os.path.join(
#         base_dir, "examples", "data", "gene_to_graph_workflow", "BDF_example_shacl.ttl"
#     ),  # Set a path for TTL serialization
#     threshold=0.001,
#     uml_figure_path=os.path.join(
#         base_dir, "examples", "data", "gene_to_graph_workflow", "BDF_example_shacl.png"
#     ),  # Set a path for diagram
# )

# # Display the UML figure
# display(
#     Image(
#         os.path.join(
#             base_dir, "examples", "data", "gene_to_graph_workflow", "BDF_example_shacl.png"
#         )
#     )
# )

- **ShEx**

In [ ]:
# Use without parameters (defaults)
# bdf.shex()

# Or use with parameters
# bdf.shex(
#     path=os.path.join(
#         base_dir, "examples", "data", "gene_to_graph_workflow", "BDF_example_shex.ttl"
#     ),  # Set a path for TTL serialization
#     threshold=0.001,
#     uml_figure_path=os.path.join(
#         base_dir, "examples", "data", "gene_to_graph_workflow", "BDF_example_shex.png"
#     ),  # Set a path for diagram
# )

# # Display the UML figure
# display(
#     Image(
#         os.path.join(base_dir, "examples", "data", "gene_to_graph_workflow", "BDF_example_shex.png")
#     )
# )

Elapsed time for the workflow, if ran in block:

In [ ]:
# end_time = time.time()
# elapsed_time = end_time - start_time
# hours = elapsed_time // 3600
# minutes = (elapsed_time % 3600) // 60
# seconds = elapsed_time % 60
# milliseconds = (elapsed_time % 1) * 1000

# # Display elapsed time in h:m:s:ms
# print(f"Elapsed Time: {int(hours)}h {int(minutes)}m {int(seconds)}s {int(milliseconds)}ms")

##### 8.5.3 Set up a virtuoso endpoint to query the RDF graph

[This repository](https://github.com/jmillanacosta/fast-virtuoso) provides a quick way to set up a local Virtuoso endpoint using its docker image.

Upload the prefixes (`bdf.shacl_prefixes(path="your/path")`) and the BDF graph generated above (`bdf.serialize("your/path", format="ttl")`).

In [ ]:
# Define a function to send some sample SELECT queries and return the response in a pandas DataFrame
# import requests


# def send_sparql_query(query, endpoint="http://localhost:8899/sparql", format="text/csv"):
#     headers = {"Accept": format}
#     params = {"query": query}
#     response = requests.get(endpoint, headers=headers, params=params)

#     # Check if request was successful
#     if response.status_code == 200:
#         if format == "text/csv":
#             # Convert CSV response to pandas DataFrame
#             from io import StringIO

#             csv_data = StringIO(response.text)
#             return pd.read_csv(csv_data)
#         else:
#             return response.text  # For other formats if needed
#     else:
#         raise Exception(f"Query failed with status code {response.status_code}: {response.text}")

In [ ]:
# from rdflib import RDF, RDFS, SH, Graph

# # Parse the query graph
# # Load the graph
# query_g = Graph()
# with open("examples/SPARQL/queries.ttl", "r") as f:
#     query_g.parse(f, format="turtle")

# # Extract queries and comments into a list of dictionaries
# queries_list = []
# for s in query_g.subjects(RDF.type, SH.SPARQLSelectExecutable):
#     query_text = query_g.value(s, SH.select)
#     comment = query_g.value(s, RDFS.comment)

#     if query_text and comment:
#         # Append a dictionary for each query and its comment
#         queries_list.append({"comment": comment.value, "query": query_text.value})

In [ ]:
# query_0 = send_sparql_query(queries_list[0]["query"])
# print(queries_list[0]["comment"])
# print(queries_list[0]["query"])
# query_0

In [ ]:
# query_3 = send_sparql_query(queries_list[3]["query"])
# print(queries_list[3]["query"])
# print(queries_list[3]["comment"])
# query_3